# Proyecto: Análisis de mercado inmobiliario

El principal objetivo de este proyecto es mejorar los resultados obtenidos en una primera instancia.
1. Mejorar los resultados en lo que respecta a código, hacerlo mas eficiente e intentar optimizar memoria del equipo.
1. Mejorar el analisis EDA para una mejor comprensión de nuestros datos.
1. Mejorar los resultados obtenidos en la predicción del modelo.
1. Incorporar un mapa iteractivo para poder visualizar el precio de las propiedades.

### Importamos herramientas a utilizar.

[Dataset properati.](https://drive.google.com/uc?export=download&id=1Ugbsw5XbNRbglomSQO1qkAgMFB_3BzmB)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
sns.set()
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso
import xgboost as xgb
from wordcloud import WordCloud, STOPWORDS 

### IMPORTAMOS DATASET

In [ ]:
properties = pd.read_csv('DS_Proyecto_01_Datos_Properati.csv')

In [ ]:
'''CAMBIAMOS NOMBRE A LAS COLUMNAS'''
propiedades = properties.rename(columns={'l1': 'country','l2':'zone','l3':'quarter'})
propiedades.head(2)

**COMENZAMOS A REALIZAR EL PREPROCESAMIENTO DE DATOS.**

1. Selecciona la zona y tipo de propiedades con mayor cantidad de propiedades.
1. Eliminamos duplicados y columnas necesarias.
1. Reemplazamos superficie cubierta mayor que superficie total con superficie total.
1. Imputamos correctamente los valores de superficie total y cubierta mal imputados por la posición del "."
1. Rellenamos null values con la mediana del valor faltante teniendo en cuenta tipo de propiedad, barrio y habitaciones.
1. Quitamos outliars mediante rango intercuartílico.
1. Aplicamos Word cloud para obtener caracteristicas relevantes de las publicaciones.
1. Aplicamos variables dummies.
1. Aplicamos reduccion de dimensionalidad.

In [ ]:
#Cantidad de propiedades por zona
sns.countplot(x = "zone", data = propiedades, dodge=False,palette= "flare",
              order = propiedades['zone'].value_counts().index)
#Formato de la gráfica
plt.title('Propiedades por zona')
plt.xlabel('Zona')
plt.ylabel('Price')

In [ ]:
#Cantidad de propiedades por zona
sns.countplot(x = "property_type", data = propiedades, dodge=False,palette= "flare",
              order = propiedades['property_type'].value_counts().index)
#Formato de la gráfica
plt.title('Tipos de propiedad')
plt.xlabel('Tipos de propiedad')
plt.ylabel('Price')

In [ ]:
'''FILTRAMOS POR ZONA Y TIPOS DE PROPIEDAD CON MAYOR CANTIDAD DE PROPIEDADES'''
ciudad_donde_hay_mas_propiedades = propiedades.zone.value_counts().idxmax()
property_type_max = list(propiedades.property_type.value_counts().nlargest(3).index)

In [ ]:
capital_federal = propiedades[propiedades.zone.isin([ciudad_donde_hay_mas_propiedades]) & propiedades.property_type.isin(property_type_max)]

In [ ]:
'''IDENTIFICAMOS DUPLICADOS Y ELIMINAMOS DUPLICADOS'''
duplicate_rows_df = capital_federal[capital_federal.duplicated()]
print('Number of duplicate rows: ', duplicate_rows_df.shape)
capital_federal = capital_federal.drop_duplicates()

In [ ]:
'''ELIMINAMOS COLUMNAS INNECESARIAS'''
capital_federal = capital_federal.drop(['start_date', 'end_date','created_on','operation_type','currency','country'], axis = 1)

In [ ]:
'''HAY PROPIEDADES CON MAYOR SUPERFICIE CUBIERTA QUE TOTAL. REEMPLAZAMOS ESTOS DATOS POR LA SUP TOTAL'''
capital_federal['surface_covered'] = np.where(capital_federal['surface_covered'] >= capital_federal.surface_total, capital_federal['surface_total'], capital_federal['surface_covered'])

**Identificamos null values**

In [ ]:
capital_federal.isna().sum()/capital_federal.shape[0]*100

In [ ]:
dat = capital_federal.isnull()
heat=sns.heatmap(dat.T, cmap='YlGnBu',xticklabels=False)
plt.title('Datos faltantes')
plt.xlabel('Instancias dataset')
plt.ylabel('Columnas dataset')
plt.tight_layout()

In [ ]:
'''REALIZAMOS BOXPLOT DEL PRECIO Y ZONA PARA IDENTIFICAR SI TENEMOS OUTLIERS. '''
sns.boxplot(x = 'price', y = 'property_type', hue= 'property_type', data = capital_federal)

In [ ]:
#Quitamos dato erroneo antes del pre procesamiento. 
capital_federal = capital_federal[capital_federal.price < 10000000]

In [ ]:
'''REALIZAMOS BOXPLOT DE LA SUPERFICIE TOTAL Y ZONA PARA IDENTIFICAR SI TENEMOS OUTLIERS. '''
sns.boxplot(x = 'surface_total', y = 'property_type', hue= 'property_type', data = capital_federal)

In [ ]:
capital_federal.surface_total = [x/100 if x>2500 else x for x in capital_federal.surface_total]

In [ ]:
'''REALIZAMOS BOXPLOT DE LA SUPERFICIE CUBIERTA Y ZONA PARA IDENTIFICAR SI TENEMOS OUTLIERS. '''
sns.boxplot(x = 'surface_covered', y = 'property_type', hue= 'property_type', data = capital_federal)

In [ ]:
capital_federal.surface_covered = [x/100 if x>2500 else x for x in capital_federal.surface_covered]

**Llenamos null values con la mediana teniendo en cuenta el tipo de propiedad, barrio y cantidad de habitaciones.**

In [ ]:
capital_federal['surface_covered'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['surface_covered'].transform(pd.Series.median), inplace = True)
capital_federal['surface_total'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['surface_total'].transform(pd.Series.median), inplace = True)
capital_federal['bathrooms'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['bathrooms'].transform(pd.Series.median).round(0), inplace = True)
capital_federal.dropna(inplace=True)

**IQR por tipo de propiedad.**

In [ ]:
#QUITAMOS VALORES ATIPICOS. ELEGIMOS RANGO INTERCUATILICO POR LA DISTRIBUCION DE LOS DATOS

departamento = capital_federal[capital_federal['property_type'] == 'Departamento']

casa = capital_federal[capital_federal['property_type'] == 'Casa']

PH = capital_federal[capital_federal['property_type'] == 'PH']


Q1_departamento = departamento.quantile(0.25).round(2)
Q3_departamento = departamento.quantile(0.75).round(2)
IQR_departamento = Q3_departamento - Q1_departamento
print('IQR de los departamentos es de:',IQR_departamento)
capital_federal_departamentos = departamento[~((departamento < (Q1_departamento - 1.5 * IQR_departamento)) |(departamento > (Q3_departamento + 1.5 * IQR_departamento))).any(axis=1)]

Q1_casa = casa.quantile(0.25).round(2)
Q3_casa = casa.quantile(0.75).round(2)
IQR_casa = Q3_casa - Q1_casa
print('IQR de las casas es de:',IQR_casa)
capital_federal_casas = casa[~((casa < (Q1_casa - 1.5 * IQR_casa)) |(casa > (Q3_casa+ 1.5 * IQR_casa))).any(axis=1)]

Q1_PH = PH.quantile(0.25).round(2)
Q3_PH = PH.quantile(0.75).round(2)
IQR_PH = Q3_PH - Q1_PH
print('IQR de los PH es de:',IQR_PH)
capital_federal_PHS = PH[~((PH < (Q1_PH - 1.5 * IQR_PH)) |(PH > (Q3_PH+ 1.5 * IQR_PH))).any(axis=1)]


capital_federal = pd.concat([capital_federal_departamentos, capital_federal_casas,capital_federal_PHS])

**Agregamos columna precio x mt2**

In [ ]:
capital_federal['mt2'] = capital_federal['price']/capital_federal['surface_total']

**Pivot by quarter and property type**

In [ ]:
cf_quarter_price = capital_federal.pivot_table(columns = ['property_type'], values = ['price'], index = ['quarter'], aggfunc = 'mean').round(2)
cf_quarter_price = cf_quarter_price.reset_index()
cf_quarter_price.head()

**Precio medio quarters**

In [ ]:
quarters_caros = capital_federal.groupby('quarter').mean().sort_values(('price'),ascending=False).round(2)
quarters_caros.head()

In [ ]:
sns.pairplot(data = capital_federal, hue='property_type')

DISTRIBUCION DEL PRECIO DE LAS PROPIEDADES

In [ ]:
sns.histplot(data = capital_federal, x = 'price', hue='property_type', color = 'skyblue')

In [ ]:
#Cantidad de pasos por tipo de vehiculo en los días de la semana.
plt.figure(figsize = (6,6))
sns.barplot(data=capital_federal, x='property_type', y ='price', palette= "flare",order = capital_federal['property_type'].value_counts().index)

#Formato de la gráfica
plt.title('Property type by price')
plt.xlabel('Property type')
plt.ylabel('Price')

In [ ]:
plt.figure(figsize = (19,7))
sns.barplot(x = "quarter",y='price', data = capital_federal, palette= "flare",order = capital_federal['quarter'].value_counts().index)
plt.title("Precio medio por barrio")
plt.ylabel("Precio")
plt.xlabel("Barrio")
plt.xticks(rotation=50)

plt.show()

**Precio medio por tipo de propiedad**

In [ ]:
capital_federal_mean_price = capital_federal.pivot_table(values = 'price', index = ['property_type'], aggfunc = 'mean').round(2)
capital_federal_mean_price = capital_federal_mean_price.reset_index()
print(capital_federal_mean_price.head())

**Superficie media por tipo de propiedad**

In [ ]:
capital_federal_mean_supercifie = capital_federal.pivot_table(values = 'surface_total', index = ['property_type'], aggfunc = 'mean').round(2)
capital_federal_mean_supercifie = capital_federal_mean_supercifie.reset_index()
print(capital_federal_mean_supercifie.head())

### Heatmap

In [ ]:
sns.heatmap(capital_federal.corr(),cmap='YlGnBu',annot=True)

### Word Cloud

In [ ]:
#pasar todos a minuscula y agregar stopwords
capital_federal['title']       = capital_federal['title'].str.lower()
capital_federal['description']       = capital_federal['description'].str.lower()
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#nltk librería de análisis de lenguaje
import nltk

#Este proceso puede hacerse antes de forma manual, descargar las stopwords de la librería nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_sp = set(stopwords.words('spanish'))
stop_words_sp = ['Venta','Departamento','ph','u','s','todas','unidad','intermediación','acceso','encuentra','encuentran','supercifies','flores','aproximadas','parte', 'superficie', 'caracteristicas', 'balvanera', 'dpto','caracteristica', 'toda', 'conclusión', 'sujetos', 'publicadas', 'consignadas', 'medidas', 'expensas', 'disponibilidad','contractual','dormitorios','publicada','urquiza','accedé','servicio','fotos','unidades','publicadas' 'accedé', 'foto', 'salida', 'sujetas', 'carácter', 'lendar','barrio','norte','baño', 'previo', 'aviso', 'cuenta', 'oportunidad', 'verificación','querés','comprá','simulá','podés','préstamo','sujeta','tipo','deja','ser','placard','cuadras','modificado','constancia','bano','inmueble','caracter', 'excelente', 'verificacion', 'n', 'queres', 'podes', 'accede', 'ajuste', 'mls', 'id','puede','dormitorio', 'ambientes','cuota','prestamo', 'p','pisos','dormitorio','precio','apto','profesional','ambiente','living','comedor','cuadra','corredor','completo','piso','simula','propiedad','responsable','dos','amb','belgrano','edificio','valor','villa','crespo','br', 'Almagro', 'Palermo', 'Villa crespo', 'casa', 'Depto', 'Caballito','Capital federal','capital', 'federal', 'Recoleta','capital federal'] + list(stop_words_sp)

text = " ".join(x for x in capital_federal.description)
print ("Hay {} palabras en el total de descripciones de los anuncios de la base".format(len(text)))

#stop_words_sp.update(["br"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stop_words_sp, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

text = " ".join(x for x in capital_federal.title)
print ("Hay {} palabras en el total de descripciones de los anuncios de la base".format(len(text)))

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stop_words_sp, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
capital_federal['title']       = capital_federal['title'].str.lower()
capital_federal['title']       = capital_federal['title'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
capital_federal['description'] = capital_federal['description'].str.lower()
capital_federal['description'] = capital_federal['description'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

capital_federal['cochera']  = ((capital_federal['description'].str.contains('cochera|garage', na=False)) | (capital_federal['title'].str.contains('cochera|garage', na=False))).astype(int)
capital_federal['balcon']   = ((capital_federal['description'].str.contains('balcon|balcón', na=False))         | (capital_federal['title'].str.contains('balcon|balcón', na=False))).astype(int)
capital_federal['frente']   = ((capital_federal['description'].str.contains('frente', na=False))         | (capital_federal['title'].str.contains('frente', na=False))).astype(int)
capital_federal['terraza']  = ((capital_federal['description'].str.contains('terraza', na=False))    | (capital_federal['title'].str.contains('terraza', na=False))).astype(int)

print('Publicaciones con cocheras:', capital_federal['cochera' ].sum())
print('Publicaciones con balcon:  ', capital_federal['balcon'  ].sum())
print('Publicaciones frente:      ', capital_federal['frente'  ].sum())
print('Publicaciones con terraza:', capital_federal['terraza' ].sum())

In [ ]:
'''CANTIDAD DE PROPIEDADES POR ZONA'''
'''CANTIDAD DE PROPIEDADES POR BARRIO '''

plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
sns.countplot(y = "property_type", hue = "property_type", data = capital_federal, dodge=False,
              order = capital_federal['property_type'].value_counts().index)
plt.title("Cantidad de anuncios por zona")
plt.xlabel("Cantidad de anuncios")
plt.ylabel("")
plt.legend("")

plt.subplot(1,2,2)
sns.countplot(y = "quarter", hue = "property_type", data = capital_federal, dodge=False,
              order = capital_federal['quarter'].value_counts().index)
plt.title("Cantidad de anuncios por barrio")
plt.xlabel("Cantidad de anuncios")
plt.ylabel("")
plt.legend(loc='lower right', frameon=False)

plt.tight_layout()

plt.show()

**Aplicamos un mapa en el que se puede observar el precio de las propiedades**

In [ ]:
# import packages, see https://python-visualization.github.io/folium/
import folium
from folium.plugins import MarkerCluster

# center to the mean of all points
mapa_capital_federal = folium.Map(location=capital_federal[["lat", "lon"]].mean().to_list(),zoom_start=15)

# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster
marker_cluster = MarkerCluster().add_to(mapa_capital_federal)

# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in capital_federal.iterrows():
    location = (r["lat"], r["lon"])
    folium.Marker(location=location,
                      popup = r['price'],
                      tooltip=r['price'])\
    .add_to(marker_cluster)

# display the map
mapa_capital_federal

**Aplicamos dummies**

In [ ]:
capital_federal = capital_federal.drop(['title', 'description','zone'], axis = 1)

In [ ]:
capital_federal = pd.get_dummies(capital_federal, columns= ['property_type','quarter'])

**Reduccion de dimensionalidad con PCA.**

In [ ]:
'''SELECCIONAMOS NUESTRAS X, Y'''
X, y = capital_federal.drop(['price'],axis=1), capital_federal['price']

**Estandarizamos nuestros datos.**

In [ ]:
scl = StandardScaler()
x = scl.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
principalComponents  = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
capital_federal = pd.concat([principalDf, capital_federal['price']], axis=1)

### Modelos ML a aplicar:
1. Como benchmark una regresión lineal.
1. Regularización con Lasso.
1. XGBoost
1. Random Forest.

**Benchmark**

In [ ]:
'''DIVIDIMOS EN TRAIN, TEST, SPLIT, ESTANDARIZAMOS Y APLICAMOS MODELO XGB'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print('Proporcion de etiquetas positiva en los datos de Train: ', y_train.sum()/y_train.size)
print('Proporcion de etiquetas positiva en los datos de Test: ', y_test.sum()/y_test.size)

Entrenamiento, predicción y errores

In [ ]:
linear_model = LinearRegression()

In [ ]:
linear_model.fit(X_train, y_train)

In [ ]:
y_train_pred_linear = linear_model.predict(X_train)
y_test_pred_linear = linear_model.predict(X_test)

In [ ]:
rmse_train_linear = np.sqrt(mean_squared_error(y_train, y_train_pred_linear))
rmse_test_linear = np.sqrt(mean_squared_error(y_test, y_test_pred_linear))
print(f'Raíz del error cuadrático medio de linear model en Train: {rmse_train_linear.round(2)}')
print(f'Raíz del error cuadrático medio de linear model en Test: {rmse_test_linear.round(2)}')

FEATURE IMPORTANCES

In [ ]:
print('The intercept is ' + str(linear_model.intercept_.round(2)))
print('The score is ' + str(linear_model.score(X_test, y_test).round(3)))
importance_lineal = linear_model.coef_.round(2)
for i,v in enumerate(importance_lineal):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
'''GRAFICO FEATURE IMPORTANCES'''
plt.bar([x for x in range(len(importance_lineal))], importance_lineal)
plt.show()

**Lasso**

Entrenamiento, predicción y errores

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso()

In [ ]:
param_dist_lasso = {'alpha': [0.1, 0.5, 1]}

In [ ]:
reg_lasso = RandomizedSearchCV(lasso, param_dist_lasso, cv=3,random_state = 42).fit(X_train,y_train)

In [ ]:
y_train_pred_lasso = reg_lasso.predict(X_train)
y_test_pred_lasso = reg_lasso.predict(X_test)

In [ ]:
rmse_trainlasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_testlasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
print(f'Raíz del error cuadrático medio en Train: {rmse_trainlasso.round(2)}')
print(f'Raíz del error cuadrático medio en Test: {rmse_testlasso.round(2)}')

FEATURE IMPORTANCES

In [ ]:
print('The intercept is ' + str(reg_lasso.best_estimator_.intercept_.round(2)))
print('The score is ' + str(reg_lasso.score(X_test, y_test).round(3)))
importance_lasso = reg_lasso.best_estimator_.coef_.round(2)
for i,v in enumerate(importance_lasso):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
'''GRAFICO FEATURE IMPORTANCES'''
plt.bar([x for x in range(len(importance_lasso))], importance_lasso)
plt.show()

### Aplicamos XGBoost y Random Forest optimizando hiperparametros.

**XGBOOST**

In [ ]:
xgb = xgb.XGBRegressor()

In [ ]:
param = {'n_estimators':[2,4,6,8,10],
         'eta': [0.1,0.5,1],
         'max_depth': [3,5,7,9,12]}

In [ ]:
model_xgb = RandomizedSearchCV(xgb, param,n_iter=10, random_state=42, cv=4).fit(X_train,y_train)

In [ ]:
'''OBSERVAMOS LOS MEJORES PARAMETROS Y EL MEJOR SCORE'''
print("Mejores parametros del modelo xgb: "+str(model_xgb.best_params_))
print("Mejor Score: "+str(model_xgb.best_score_.round(3))+'\n')

In [ ]:
'''REALIZAMOS LAS PREDICCIONES'''
y_train_pred_xgb = model_xgb.predict(X_train)
y_test_pred_xgb = model_xgb.predict(X_test)

In [ ]:
'''EVALUAMOS MEDIANTE EL ERROR CUADRATICO MEDIO, METRICA UTILIZADA A LO LARGO DEL PROYECTO PARA COMPARAR RESULTADOS.'''
rmse_train_xgb = np.sqrt(mean_squared_error(y_train, y_train_pred_xgb))
rmse_test_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
print(f'Raíz del error cuadrático medio en Train xgb: {rmse_train_xgb.round(2)}')
print(f'Raíz del error cuadrático medio en Test xgb: {rmse_test_xgb.round(2)}')

FEATURE IMPORTANCES

In [ ]:
importances_xgb = model_xgb.best_estimator_.feature_importances_.round(3)
for i,v in enumerate(importances_xgb):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
'''GRAFICO FEATURE IMPORTANCES'''
plt.bar([x for x in range(len(importances_xgb))], importances_xgb)
plt.show()

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor()

In [ ]:
'''HIPERPARAMETROS DE NUESTRO RANDOM FOREST'''
param = {'max_depth': [13,15,20,25],
         'n_estimators' : [80],
        'max_features': ['auto', 'sqrt', 'log2']}

In [ ]:
model_forest = RandomizedSearchCV(forest, param,n_iter=10, random_state=42, cv=4, n_jobs = -1).fit(X_train,y_train)

In [ ]:
print("Mejores parametros del modelo random forest: "+str(model_forest.best_params_))
print("Mejor Score: "+str(model_forest.best_score_.round(3))+'\n')

In [ ]:
'''REALIZAMOS PREDICCIONES'''
y_train_pred_forest = model_forest.predict(X_train) 
y_test_pred_forest = model_forest.predict(X_test)

In [ ]:
'''EVALUAMOS MEDIANTE EL ERROR CUADRATICO MEDIO, METRICA UTILIZADA A LO LARGO DEL PROYECTO PARA COMPARAR RESULTADOS.'''
rmse_train_forest = np.sqrt(mean_squared_error(y_train, y_train_pred_forest))
rmse_test_forest = np.sqrt(mean_squared_error(y_test, y_test_pred_forest))
print(f'Raíz del error cuadrático medio en Train random forest: {rmse_train_forest.round(2)}')
print(f'Raíz del error cuadrático medio en Test random forest: {rmse_test_forest.round(2)}')

FEATURE IMPORTANCES

In [ ]:
forest_importances = model_forest.best_estimator_.feature_importances_.round(3)
for i,v in enumerate(forest_importances):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
'''GRAFICO FEATURE IMPORTANCES'''
plt.bar([x for x in range(len(forest_importances))], forest_importances)
plt.show()

In [ ]:
'''Best estimator of the forest 48'''
tree_48 = model_forest.best_estimator_.estimators_[47]
print(tree_48)

**Graficamos los errores de cada uno de los modelos aplicados.**

In [ ]:
'''GRAFICAMOS LOS ERRORES, LA DISPERSIÓN DE CADA UNO DE LOS MODELOS ANALIZADOS.'''
modelos = ['Linear Model', 'Regularization lasso', 'XGBoost', 'Random Forest']

for i, model in enumerate([linear_model,reg_lasso, model_xgb, model_forest]):
    for pred_train, y_train_pred in enumerate([y_train_pred_linear,y_train_pred_lasso,y_train_pred_xgb,y_train_pred_forest]):
        pred_train = model.predict(X_train)
        for pred_test, y_test_pred in enumerate([y_test_pred_linear,y_test_pred_lasso,y_test_pred_xgb,y_test_pred_forest]): 
            pred_test = model.predict(X_test)
            for rmse_train in enumerate([rmse_train_linear, rmse_trainlasso, rmse_train_xgb, rmse_train_forest]):
                rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
                for rmse_test in enumerate([rmse_test_linear, rmse_testlasso, rmse_test_xgb, rmse_test_forest]):
                    rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))    
    
    print(f'Modelo: {modelos[i]}')
    rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (8,4))

    plt.subplot(1,2,1)
    sns.distplot(y_train - pred_train, bins = 20, label = 'train')
    sns.distplot(y_test - pred_test, bins = 20, label = 'test')
    plt.xlabel('errores')
    plt.legend()

    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,pred_test, s =2)    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    
    plt.tight_layout()
    plt.show()

**Luego de haber realizado el entrenamiento de todos los modelos y observar resultados concluimos que:**

El modelo ha mejorado considerablemente su predicción respecto del proyecto anterior.
   1. Consideramos que esto se da por:
    1. Limpieza mas detallada en el dataset.
        1. En lo que respecta a limpieza, hemos encontrado que en muchas propiedades, en las columnas surface covered y surface total, la imputación de los m2 estaban erronea el **"."**. Para subsanar este error he dividido por 100 a estas superficies para que queden correctamente imputadas y no me generen un desvío en los datos. Al realizar esto, el llenado de null values con la mediana es mas precisa, el IQR aplicado de igual manera.
    1. Cambio  en llenado de null values.
    1. Dummies aplicados.

**Evaluación de modelos**
1. Se puede observar que el último modelo aplicado "Random Forest" está overfited, por lo que lo descartamos.
1. El XGBoost que obtenemos un resultado mas realista y no overfiteado.

## Incorporamos información de una fuente externa.
Es el precio medio de los distintos barrios de CABA según la BBDD de la Universidad de San Andres.
Es información simplemente la vamos a utilizar para comparar la media de precios de ambas BBDD.

[Descarga de los datos aquí.](https://udesa.edu.ar/indices-meli/ventas). Barrios CABA

In [ ]:
#Descargamos el dataset y lo leemos

In [ ]:
caba = pd.read_excel(('ventascaba_202105.xlsx'),
     engine='openpyxl')

In [ ]:
properties = pd.read_csv('DS_Proyecto_01_Datos_Properati.csv')

In [ ]:
caba.head()

Pasamos a miniscula el nombre de los barrio para poder unirlos

In [ ]:
print(caba['Barrio'].unique())

In [ ]:
print(propiedades['quarter'].unique())

In [ ]:
caba["Barrio"].map(lambda Barrio: Barrio.lower())

In [ ]:
def upcase_first_letter(s):
    return s[0].upper() + s[1:]

In [ ]:
#Tabla pivot de la media
barrios = caba.pivot_table(index = ['Barrio'],values = 'Mediana Flujo', aggfunc = 'mean')
barrios = barrios.reset_index()
barrios.head()

In [ ]:
#Cantidad de propiedades por barrio
sns.countplot(x = "Barrio", data = caba, dodge=False,palette= "flare",
              order = caba['Barrio'].value_counts().index)
#Formato de la gráfica
plt.title('Propiedades por barrio')
plt.xlabel('Barrio')
plt.ylabel('Price')

In [ ]:
#Se importa de nuevo la BBDD de properati y realizamos la misma limpieza que para el modelado.

In [ ]:
'''CAMBIAMOS NOMBRE A LAS COLUMNAS'''
propiedades = properties.rename(columns={'l1': 'country','l2':'zone','l3':'quarter'})

In [ ]:
'''FILTRAMOS POR ZONA Y TIPOS DE PROPIEDAD CON MAYOR CANTIDAD DE PROPIEDADES'''
ciudad_donde_hay_mas_propiedades = propiedades.zone.value_counts().idxmax()
property_type_max = list(propiedades.property_type.value_counts().nlargest(3).index)

In [ ]:
capital_federal = propiedades[propiedades.zone.isin([ciudad_donde_hay_mas_propiedades]) & propiedades.property_type.isin(property_type_max)]

In [ ]:
'''IDENTIFICAMOS DUPLICADOS Y ELIMINAMOS DUPLICADOS'''
duplicate_rows_df = capital_federal[capital_federal.duplicated()]
print('Number of duplicate rows: ', duplicate_rows_df.shape)
capital_federal = capital_federal.drop_duplicates()

In [ ]:
'''ELIMINAMOS COLUMNAS INNECESARIAS'''
capital_federal = capital_federal.drop(['start_date', 'end_date','created_on','operation_type','currency','country'], axis = 1)

In [ ]:
'''HAY PROPIEDADES CON MAYOR SUPERFICIE CUBIERTA QUE TOTAL. REEMPLAZAMOS ESTOS DATOS POR LA SUP TOTAL'''
capital_federal['surface_covered'] = np.where(capital_federal['surface_covered'] >= capital_federal.surface_total, capital_federal['surface_total'], capital_federal['surface_covered'])

In [ ]:
#Quitamos dato erroneo antes del pre procesamiento. 
capital_federal = capital_federal[capital_federal.price < 10000000]
capital_federal.surface_total = [x/100 if x>2500 else x for x in capital_federal.surface_total]
capital_federal.surface_covered = [x/100 if x>2500 else x for x in capital_federal.surface_covered]

In [ ]:
capital_federal['surface_covered'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['surface_covered'].transform(pd.Series.median), inplace = True)
capital_federal['surface_total'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['surface_total'].transform(pd.Series.median), inplace = True)
capital_federal['bathrooms'].fillna(capital_federal.groupby(['property_type','quarter','rooms','bedrooms'])['bathrooms'].transform(pd.Series.median).round(0), inplace = True)
capital_federal.dropna(inplace=True)

In [ ]:
#QUITAMOS VALORES ATIPICOS. ELEGIMOS RANGO INTERCUATILICO POR LA DISTRIBUCION DE LOS DATOS

departamento = capital_federal[capital_federal['property_type'] == 'Departamento']

casa = capital_federal[capital_federal['property_type'] == 'Casa']

PH = capital_federal[capital_federal['property_type'] == 'PH']


Q1_departamento = departamento.quantile(0.25).round(2)
Q3_departamento = departamento.quantile(0.75).round(2)
IQR_departamento = Q3_departamento - Q1_departamento
#print('IQR de los departamentos es de:',IQR_departamento)
capital_federal_departamentos = departamento[~((departamento < (Q1_departamento - 1.5 * IQR_departamento)) |(departamento > (Q3_departamento + 1.5 * IQR_departamento))).any(axis=1)]

Q1_casa = casa.quantile(0.25).round(2)
Q3_casa = casa.quantile(0.75).round(2)
IQR_casa = Q3_casa - Q1_casa
#print('IQR de las casas es de:',IQR_casa)
capital_federal_casas = casa[~((casa < (Q1_casa - 1.5 * IQR_casa)) |(casa > (Q3_casa+ 1.5 * IQR_casa))).any(axis=1)]

Q1_PH = PH.quantile(0.25).round(2)
Q3_PH = PH.quantile(0.75).round(2)
IQR_PH = Q3_PH - Q1_PH
#print('IQR de los PH es de:',IQR_PH)
capital_federal_PHS = PH[~((PH < (Q1_PH - 1.5 * IQR_PH)) |(PH > (Q3_PH+ 1.5 * IQR_PH))).any(axis=1)]


capital_federal = pd.concat([capital_federal_departamentos, capital_federal_casas,capital_federal_PHS])

In [ ]:
capital_federal['mt2'] = capital_federal['price']/capital_federal['surface_total']

In [ ]:
#Unimos las dos BBDD

In [ ]:
df_merge = pd.merge(capital_federal, barrios ,left_on="quarter", right_on="Barrio")
df_merge.shape

In [ ]:
df_merge = df_merge.drop(['Barrio'],axis=1)

In [ ]:
#Graficamos la media del precio de mt2 según la BBDD de properati y la de San Andres.

In [ ]:
#Tabla pivot de la media
comparativa = df_merge.pivot_table(index = ['quarter'],values = ['mt2','Mediana Flujo']).round(2)
comparativa = comparativa.reset_index()
comparativa

In [ ]:
#Argegamos columna para distinguir cual es mayor
df_merge['comparativa'] = np.where(df_merge["mt2"] >= df_merge["Mediana Flujo"], 1, 0)
df_merge.comparativa

Grafico que compara la media de precio en cada barrio según cada BBDD

In [ ]:
plt.figure(figsize = (19,7))
sns.countplot(x = "quarter",hue='comparativa', data = df_merge, palette= "flare",order = df_merge['quarter'].value_counts().index)
plt.title("Precio medio por barrio")
plt.ylabel("Precio mt2")
plt.xlabel("Barrio")
plt.xticks(rotation=45)
plt.legend(loc='lower right', frameon=False)

plt.tight_layout()

plt.show()